##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Tensorflow Ranking Keras pipeline for distributed training

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/ranking/tutorials/ranking_dnn_distributed"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/ranking/blob/master/docs/tutorials/ranking_dnn_distributed.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/ranking/blob/master/docs/tutorials/ranking_dnn_distributed.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/ranking/docs/tutorials/ranking_dnn_distributed.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

TensorFlow Ranking can handle heterogeneous dense and sparse features, and scales up to millions of data points. However, building and deploying a learning to rank model to operate at scale creates additional challenges beyond simply designing a model. The Ranking library provides workflow utility classes for building [distributed training](https://www.tensorflow.org/guide/distributed_training) for large-scale ranking applications. For more information about these features, see the TensorFlow Ranking [Overview](https://www.tensorflow.org/ranking/overview).

This tutorial shows you how to build a ranking model that enables a distributed processing strategy by using the Ranking library's support for a pipeline processing architecture.

Note: An advanced version of this code is also available as a [Python script](https://github.com/tensorflow/ranking/blob/master/tensorflow_ranking/examples/keras/antique_ragged.py). The script version supports flags for hyperparameters, and advanced use-cases like [Document Interaction Network](https://research.google/pubs/pub49364).

## ANTIQUE dataset

In this tutorial, you will build a ranking model for ANTIQUE, a question-answering dataset. Given a query, and a list of answers, the objective is to rank the answers with optimal rank related metrics, such as NDCG. For more details about ranking metrics, review evaluation measures [offline metrics](https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)#Offline_metrics).

[ANTIQUE](https://ciir.cs.umass.edu/downloads/Antique/) is a publicly available dataset for open-domain non-factoid question answering, collected over Yahoo! answers.
Each question has a list of answers, whose relevance are graded on a scale of 0-4, 0 for irrelevant and 4 for fully relevant.
The list size can vary depending on the query, so we use a fixed "list size" of 50, where the list is either truncated or padded with default values.
The dataset is split into 2206 queries for training and 200 queries for testing. For more details, please read the technical paper on [arXiv](https://arxiv.org/abs/1905.08957).

## Setup

Download and install the TensorFlow Ranking and TensorFlow Serving packages.

In [ ]:
!pip install -q tensorflow-ranking tensorflow-serving-api

Import TensorFlow Ranking library and useful libraries through the notebook.

In [ ]:
import pathlib

import tensorflow as tf
import tensorflow_ranking as tfr
from tensorflow_serving.apis import input_pb2
from google.protobuf import text_format

## Data preparation

Download training, test data, and vocabulary file.

In [ ]:
!wget -O "/tmp/train.tfrecords" "http://ciir.cs.umass.edu/downloads/Antique/tf-ranking/ELWC/train.tfrecords"
!wget -O "/tmp/test.tfrecords" "http://ciir.cs.umass.edu/downloads/Antique/tf-ranking//ELWC/test.tfrecords"
!wget -O "/tmp/vocab.txt" "http://ciir.cs.umass.edu/downloads/Antique/tf-ranking/vocab.txt"

Here, the dataset is saved in a ranking-specific ExampleListWithContext (ELWC) format. Detailed in the next section, shows how to generate and store data in the ELWC format.

### ELWC Data Formats for Ranking

The data for a single question consists of a list of `query_tokens` representing the question (the "context"), and a list of answers (the "examples"). Each answer is represented as a list of `document_tokens` and a `relevance` score. You could represent an example question like this:

In [ ]:
example_list_with_context = {
    "context": {
        "query_tokens": ["this", "is", "a", "question"]
    },
    "examples": [
        {
            "document_tokens": ["this", "is", "a", "relevant", "answer"],
            "relevance": [4]
        },
        {
            "document_tokens": ["irrelevant", "data"],
            "relevance": [0]
        }
    ]
}

The data files, downloaded in the previous section, contain a serialized [protobuffer](https://developers.google.com/protocol-buffers/) representation of this sort of data.
These protobuffers are quite long when viewed as text, but encode the same data.

In [ ]:
CONTEXT = text_format.Parse(
    """
    features {
      feature {
        key: "query_tokens"
        value { bytes_list { value: ["this", "is", "a", "question"] } }
      }
    }""", tf.train.Example())

In [ ]:
EXAMPLES = [
    text_format.Parse(
    """
    features {
      feature {
        key: "document_tokens"
        value { bytes_list { value: ["this", "is", "a", "relevant", "answer"] } }
      }
      feature {
        key: "relevance"
        value { int64_list { value: 4 } }
      }
    }""", tf.train.Example()),
    text_format.Parse(
        """
    features {
      feature {
        key: "document_tokens"
        value { bytes_list { value: ["irrelevant", "data"] } }
      }
      feature {
        key: "relevance"
        value { int64_list { value: 0 } }
      }
    }""", tf.train.Example()),
]

In [ ]:
ELWC = input_pb2.ExampleListWithContext()
ELWC.context.CopyFrom(CONTEXT)
for example in EXAMPLES:
  example_features = ELWC.examples.add()
  example_features.CopyFrom(example)

print(ELWC)

While the text format is verbose, protos can be efficiently serialized to a byte string (and parsed back into a proto)

In [ ]:
serialized_elwc = ELWC.SerializeToString()
print(serialized_elwc)

The following parser configuration parses the binary representation into a dictionary of tensors:

In [ ]:
def parse_elwc(elwc):
  return tfr.data.parse_from_example_list(
      [elwc],
      list_size=2,
      context_feature_spec={"query_tokens": tf.io.RaggedFeature(dtype=tf.string)},
      example_feature_spec={
          "document_tokens":
              tf.io.RaggedFeature(dtype=tf.string),
          "relevance":
              tf.io.FixedLenFeature(shape=[], dtype=tf.int64, default_value=0)
      },
      size_feature_name="_list_size_",
      mask_feature_name="_mask_")

parse_elwc(serialized_elwc)

Note with ELWC, you could also generate `size` and/or `mask` features to indicate the valid size and/or to mask out the valid entries in the list as long as `size_feature_name` and/or `mask_feature_name` are defined.

The above parser is defined in `tfr.data` and wrapped in our predefined dataset builder `tfr.keras.pipeline.BaseDatasetBuilder`.

## Overview of the ranking pipeline

Follow the steps depicted in the figure below to train a ranking model with ranking pipeline. In particular, this example uses the `tfr.keras.model.FeatureSpecInputCreator` and `tfr.keras.pipeline.BaseDatasetBuilder` defined specific for the datasets with `feature_spec`.

![TF02  TF-Ranking_ Learning-to-rank in Tensorflow](https://user-images.githubusercontent.com/18746174/136829491-1d76c2e0-6779-4b7f-9a8c-515acc17d0ec.png)


## Create a model builder

Instead of directly building a `tf.keras.Model` object, create a model_builder, which is called in the ranking pipeline to build the `tf.keras.Model`, as all training parameters must be defined under the `strategy.scope` (called in `train_and_validate` function in ranking pipeline) in order to train with distributed strategies.

This framework uses the [keras functional api](https://www.tensorflow.org/guide/keras/functional) to build models, where inputs (`tf.keras.Input`), preprocessors (`tf.keras.layers.experimental.preprocessing`), and scorer (`tf.keras.Sequential`) are required to define the model.

### Specify Features

[Feature Specification](https://www.tensorflow.org/api_docs/python/tf/io) are TensorFlow abstractions that are used to capture rich information about each feature.

Create feature specifications for context features, example features, and labels, consistent with the input formats for ranking, such as ELWC format.

The `default_value` of `label_spec` feature is set to -1 to take care of the padding items to be masked out.

In [ ]:
context_feature_spec = {
    "query_tokens": tf.io.RaggedFeature(dtype=tf.string),
}
example_feature_spec = {
    "document_tokens":
        tf.io.RaggedFeature(dtype=tf.string),
}
label_spec = (
    "relevance",
    tf.io.FixedLenFeature(shape=(1,), dtype=tf.int64, default_value=-1)
)

### Define `input_creator`

`input_creator` create dictionaries of context and example `tf.keras.Input`s for input features defined in `context_feature_spec` and `example_feature_spec`.

In [ ]:
input_creator = tfr.keras.model.FeatureSpecInputCreator(
    context_feature_spec, example_feature_spec)

Callling the `input_creator` returns the dictionaries of Keras-Tensors, that are used as the inputs when building the model:

In [ ]:
input_creator()

### Define `preprocessor`

In the `preprocessor`, the input tokens are converted to a one-hot vector through the String Lookup preprocessing layer and then embeded as an embedding vector through the Embedding preprocessing layer. Finally, compute an embedding vector for the full sentence by the average of token embeddings.

In [ ]:
_VOCAB_FILE = '/tmp/vocab.txt'
_VOCAB_SIZE = len(pathlib.Path(_VOCAB_FILE).read_text().split())

lookup = tf.keras.layers.StringLookup(
    max_tokens=_VOCAB_SIZE,
    vocabulary=_VOCAB_FILE,
    mask_token=None)

embedding = tf.keras.layers.Embedding(
    input_dim=_VOCAB_SIZE,
    output_dim=20,
    embeddings_initializer=None,
    embeddings_constraint=None)

Note that the vocabulary uses the same tokenizer that BERT does. One could use [BertTokenizer](https://www.tensorflow.org/text/guide/subwords_tokenizer) to tokenize the raw sentences.

In [ ]:
tokenizer = tf_text.BertTokenizer(_VOCAB_FILE)
example_tokens = tokenizer.tokenize("Hello TensorFlow!".lower())

print(example_tokens)
print(tokenizer.detokenize(example_tokens))

In [ ]:
def embedding_fn(input_tokens):
  return tf.reduce_mean(embedding(lookup(input_tokens)), axis=-2)

In [ ]:
preprocessor = tfr.keras.model.PreprocessorWithSpec(
    {"query_tokens": embedding_fn, "document_tokens": embedding_fn})

### Define `scorer`

This example uses a Deep Neural Network (DNN) univariate scorer, predefined in TensorFlow Ranking.

In [ ]:
scorer = tfr.keras.model.DNNScorer(
    hidden_layer_dims=[64, 32, 16],
    output_units=1,
    activation=tf.nn.relu,
    use_batch_norm=True)

### Make `model_builder`

In addition to `input_creator`, `preprocessor`, and `scorer`, specify the mask feature name to take the mask feature generated in datasets.

In [ ]:
model_builder = tfr.keras.model.ModelBuilder(
    input_creator=input_creator,
    preprocessor=preprocessor,
    scorer=scorer,
    mask_feature_name="example_list_mask",
    name="antique_model",
)

Check the model architecture,

In [ ]:
model = model_builder.build()
tf.keras.utils.plot_model(model, expand_nested=True)

## Create a dataset builder

A `dataset_builder` is designed to create datasets for training and validation and to define [signatures](https://www.tensorflow.org/guide/saved_model#specifying_signatures_during_export) for exporting trained model as `tf.function`. 

### Specify data hyperparameters

Define the hyperparameters to be used to build datasets in `dataset_builder` by creating a `dataset_hparams` object.

Load training dataset at `/tmp/train.tfrecords` with `tf.data.TFRecordDataset` reader. In each batch, each feature tensor has a shape `(batch_size, list_size, feature_sizes)` with `batch_size` equal to 32 and `list_size` equal to 50. Validate with the test data at `/tmp/test.tfrecords` at the same `batch_size` and `list_size`.

In [ ]:
dataset_hparams = tfr.keras.pipeline.DatasetHparams(
    train_input_pattern="/tmp/train.tfrecords",
    valid_input_pattern="/tmp/test.tfrecords",
    train_batch_size=32,
    valid_batch_size=32,
    list_size=50,
    dataset_reader=tf.data.TFRecordDataset)

### Make `dataset_builder`

TensorFlow Ranking provides a pre-defined `SimpleDatasetBuilder` to generate datasets from ELWC using `feature_spec`s. As a mask feature is used to determine valid examples in each padded list, must specify the `mask_feature_name` consistent with the `mask_feature_name` used in `model_builder`.

In [ ]:
dataset_builder = tfr.keras.pipeline.SimpleDatasetBuilder(
    context_feature_spec,
    example_feature_spec,
    mask_feature_name="example_list_mask",
    label_spec=label_spec,
    hparams=dataset_hparams)

In [ ]:
ds_train = dataset_builder.build_train_dataset()
ds_train.element_spec

## Create a ranking pipeline

A `ranking_pipeline` is an optimized ranking model training package that implement distributed training, export model as `tf.function`, and integrate useful callbacks including tensorboard and restoring upon failures.

### Specify pipeline hyperparameters

Specify the hyperparameters to be used to run the pipeline in `ranking_pipeline` by creating a `pipeline_hparams` object. 

Train the model with `approx_ndcg_loss` at learning rate equal to 0.05 for 5 epoch with 1000 steps in each epoch using `MirroredStrategy`. Evaluate the model on the validation dataset for 100 steps after each epoch. Save the trained model under `/tmp/ranking_model_dir`.

In [ ]:
pipeline_hparams = tfr.keras.pipeline.PipelineHparams(
    model_dir="/tmp/ranking_model_dir",
    num_epochs=5,
    steps_per_epoch=1000,
    validation_steps=100,
    learning_rate=0.05,
    loss="approx_ndcg_loss",
    strategy="MirroredStrategy")

### Define `ranking_pipeline`

TensorFlow Ranking provides a pre-defined `SimplePipeline` to support model training with distributed strategies.

In [ ]:
ranking_pipeline = tfr.keras.pipeline.SimplePipeline(
    model_builder,
    dataset_builder=dataset_builder,
    hparams=pipeline_hparams)

## Train and evaluate the model

`train_and_validate` will evaluate the trained model on the validation dataset after every epoch.

In [ ]:
ranking_pipeline.train_and_validate(verbose=1)

### Launch TensorBoard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir="/tmp/ranking_model_dir" --port 12345

A sample tensorboard output is shown here, with the ranking metrics.

![tensorboard](https://user-images.githubusercontent.com/18746174/136845677-8cd41b8f-0a1a-4b38-b905-779966839e5f.png)


### Generate predictions and evaluate

Get test data.

In [ ]:
ds_test = dataset_builder.build_valid_dataset()

# Get input features from the first batch of the test data
for x, y in ds_test.take(1):
  break

Load the save model and predict.

In [ ]:
loaded_model = tf.keras.models.load_model("/tmp/ranking_model_dir/export/latest_model")

In [ ]:
# Predict ranking scores
scores = loaded_model.predict(x)

# Sort the answers by scores
sorted_answers = tfr.utils.sort_by_scores(
    scores,
    [tf.strings.reduce_join(x['document_tokens'], -1, separator=' ')])[0]

Check the top 5 answers for question number 4.

In [ ]:
question = tf.strings.reduce_join(
    x['query_tokens'][4, :], -1, separator=' ').numpy()
top_answers = sorted_answers[4, :5].numpy()

print(
    f'Q: {question.decode()}\n' +
    '\n'.join([f'A{i+1}: {ans.decode()}' for i, ans in enumerate(top_answers)]))